#### Importación de las noticias desde el motor de sophia

In [23]:
import elasticsearch
from datetime import datetime
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
nlp = spacy.load("es_core_news_md")

In [2]:
#Parámetros de búsqueda
country="chile"
from_="2020-01-01"
to_="2020-02-01"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

simple_keyword=False

In [4]:
#Parámetros de autentificación
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "AxahEIWKsagna321"

In [5]:
comunas= ["Puerto Montt", "Calbuco", "Cochamó", "Fresia", "Frutillar", "Los Muermos", "Llanquihue", "Maullín", "Puerto Varas", "Castro", "Ancud", "Chonchi", "Curaco de Vélez", "Dalcahue", "Puqueldón", "Queilén", "Quellón", "Quemchi", "Quinchao", "Osorno", "Puerto Octay", "Purranque", "Puyehue", "Río Negro", "San Juan de la Costa", "San Pablo", "Chaitén", "Futaleufú", "Hualaihué", "Palena"]


In [6]:
es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [7]:
#NO MODIFICAR
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [24]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 5923 noticias encontradas...


***Creamos el DataFrame***

In [28]:

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

for hit in res['hits']['hits']:

    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    df = pd.concat([df, pd.DataFrame.from_records([new_row])])


df = df.reset_index(drop=True) 

df = df.drop_duplicates(subset='url', keep='first')


In [29]:
df

,id_news,country,media_outlet,url,title,text,date
0,21892281.0,chile,radiosago,https://www.radiosago.cl/91469-2/,Cámara de Comercio de Puerto Montt cuestionó c...,"El gremio criticó que las bases de Sercotec, a...",2020-01-31
1,21892289.0,chile,radiosago,https://www.radiosago.cl/video-puerto-montt-lo...,FOTOS Y VIDEO: PUERTO MONTT: Los efectos de nu...,El corazón del sector céntrico de la ciudad de...,2020-01-30
2,21892293.0,chile,radiosago,https://www.radiosago.cl/solicitaron-pericia-p...,Solicitaron pericia psiquiátrica para mujer qu...,La joven madre de 22 años fue formalizada en e...,2020-01-29
3,21892295.0,chile,radiosago,https://www.radiosago.cl/camara-de-comercio-de...,Cámara de Comercio de Puerto Montt condenó hec...,Tras la violenta jornada registrada en la capi...,2020-01-30
4,21892296.0,chile,radiosago,https://www.radiosago.cl/formalizaran-a-oficia...,Formalizarán a oficial de carabineros acusado ...,El ministerio Público solicitó al Juzgado de G...,2020-01-30
...,...,...,...,...,...,...,...
5788,47473028.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Invitan a taller sobre postulación al fondo de...,Hasta el 29 de febrero se encuentra abierta la...,2020-01-23
5792,47472588.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Motonave auxilió a lancha que quedó al garete ...,Un desperfecto mecánico aparece como la causa ...,2020-01-22
5794,47472590.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Artefacto movilizó a equipo del GOPE a barrio ...,Un artefacto sospechoso movilizó a los servici...,2020-01-22
5796,47472593.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,denuncia y ayuda,ATROPELLO de pudú EN LA RUTA. Una vez más un e...,2020-01-22


In [32]:
# Guardamos el archivo sin las comunas agregadas por si nos sirve
nombre_archivo="DataFrame_Sin_Comunas"+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./data/"+nombre_archivo)

El archivo se llama:
DataFrame_Sin_Comunas.csv


In [33]:
#Agregamos las columnas con las comunas
#Si hay noticias con comunas "Sin especificar", simplemente no las tomamos
matcher_comunas = PhraseMatcher(nlp.vocab)
for comuna in comunas:
    matcher_comunas.add(comuna, [nlp(comuna)])

for index,row in df.iterrows():
    txt = row["text"]
    
    try:

        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end] 

    except:
        pass

In [34]:
dfComunas

,id_news,country,media_outlet,url,title,text,date,Ancud,Calbuco,Castro,...,Puqueldón,Purranque,Puyehue,Queilén,Quellón,Quemchi,Quinchao,Río Negro,San Juan de la Costa,San Pablo
0,21892281.0,chile,radiosago,https://www.radiosago.cl/91469-2/,Cámara de Comercio de Puerto Montt cuestionó c...,"El gremio criticó que las bases de Sercotec, a...",2020-01-31,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21892289.0,chile,radiosago,https://www.radiosago.cl/video-puerto-montt-lo...,FOTOS Y VIDEO: PUERTO MONTT: Los efectos de nu...,El corazón del sector céntrico de la ciudad de...,2020-01-30,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21892293.0,chile,radiosago,https://www.radiosago.cl/solicitaron-pericia-p...,Solicitaron pericia psiquiátrica para mujer qu...,La joven madre de 22 años fue formalizada en e...,2020-01-29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21892295.0,chile,radiosago,https://www.radiosago.cl/camara-de-comercio-de...,Cámara de Comercio de Puerto Montt condenó hec...,Tras la violenta jornada registrada en la capi...,2020-01-30,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21892296.0,chile,radiosago,https://www.radiosago.cl/formalizaran-a-oficia...,Formalizarán a oficial de carabineros acusado ...,El ministerio Público solicitó al Juzgado de G...,2020-01-30,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5788,47473028.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Invitan a taller sobre postulación al fondo de...,Hasta el 29 de febrero se encuentra abierta la...,2020-01-23,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5792,47472588.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Motonave auxilió a lancha que quedó al garete ...,Un desperfecto mecánico aparece como la causa ...,2020-01-22,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
5794,47472590.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,Artefacto movilizó a equipo del GOPE a barrio ...,Un artefacto sospechoso movilizó a los servici...,2020-01-22,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5796,47472593.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2020/0...,denuncia y ayuda,ATROPELLO de pudú EN LA RUTA. Una vez más un e...,2020-01-22,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Guardamos el archivo con las comunas agregadas 
nombre_archivo="DataFrame_Comunas"+".csv"
print("El archivo se llama:")
print(nombre_archivo)
dfComunas.to_csv("./data/"+nombre_archivo)

El archivo se llama:
DataFrame_Comunas.csv
